In [1]:
!pip install onboard.client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 KB 477.9 kB/s eta 0:00:00
  Attempting uninstall: orjson
    Found existing installation: orjson 3.6.7
    Uninstalling orjson-3.6.7:
      Successfully uninstalled orjson-3.6.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.37.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.4 which is incompatible.
apache-beam 2.37.0 requires httplib2<0.20.0,>=0.8, but you have httplib2 0.20.4 which is incompatible.
apache-beam 2.37.0 requires pyarrow<7.0.0,>=0.15.1, but you have pyarrow 7.0.0 which is incompatible.


In [2]:
# RTEM APIs
import requests
import pytz
from onboard.client import RtemClient
from onboard.client.models import TimeseriesQuery, PointData, PointSelector
from onboard.client.dataframes import points_df_from_streaming_timeseries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta
plt.style.use('ggplot')

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix

from IPython.display import display

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/rtem-processed/retem_all_points_metadata-2.csv
/kaggle/input/rtemsampledata/stats_merged.csv
/kaggle/input/rtemsampledata/rtem_zone_temp_time_series.csv
/kaggle/input/rtemsampledata/rtem_all_points_metadata.csv
/kaggle/input/rtemsampledata/zone_temp_time_series.csv
/kaggle/input/building-metadata/building_meta.csv


In [3]:
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (20,9)

In [4]:
client = RtemClient(api_key="...(enter your RTEM API key here)")

## API Data

# `1.0 DATA STRUCTURE`

### Equipment Types

In [5]:
# Get the types, this will return a json
types = client.get_equipment_types()

# Convert it to dataframe
equip_type = pd.json_normalize(types)

# Check it out
equip_type.head()

,id,tag_name,name_long,name_abbr,active,flow_order,critical_point_types,sub_types,tags
0,12,ahu,Air Handling Unit,AHU,True,2,"[79, 85, 103, 159, 160]","[{'id': 1, 'equipment_type_id': 12, 'tag_name'...","[hvac, ahu]"
1,19,boiler,Boiler,BLR,True,1,"[88, 131, 92, 167, 121]","[{'id': 4, 'equipment_type_id': 19, 'tag_name'...","[hvac, boiler]"
2,20,chilledWaterPlant,Chilled Water Plant,CHWS,True,1,"[89, 132, 93, 116, 108]",[],"[hvac, chilledWaterPlant]"
3,21,chiller,Chiller,CH,True,1,"[89, 132, 93, 167, 121]","[{'id': 7, 'equipment_type_id': 21, 'tag_name'...","[hvac, chiller]"
4,22,condenser,Condenser,COND,True,1,[121],[],"[hvac, condenser]"


### Point Types

In [6]:
# Get all point types from the Data Model
point_type = pd.DataFrame(client.get_all_point_types())
point_type

,id,tag_name,active,measurement_id,tag_set_ids,tags,default_unit_id
0,124,Occupied Heating Setpoint,True,1,"[15, 2, 4, 62, 51, 55]","[air, sp, temp, zone, heating, occ]",None
1,118,Outside Air Carbon Dioxide,True,6,"[15, 8, 1, 24]","[air, co2, sensor, outside]",None
2,130,Return Air Temperature Setpoint,True,1,"[15, 2, 4, 63]","[air, sp, temp, return]",None
3,84,Dual-Temp Coil Discharge Air Temperature,True,1,"[15, 36, 47, 1, 4, 250]","[air, discharge, dualTemp, sensor, temp, coil]",None
4,81,Reheat Coil Discharge Air Temperature,True,1,"[15, 36, 44, 1, 4, 250]","[air, discharge, reheats, sensor, temp, coil]",None
...,...,...,...,...,...,...,...
647,754,Evaporator Water Return Temperature,True,1,"[16, 40, 1, 4, 98]","[water, entering, sensor, temp, evaporator]",None
648,755,Evaporator Water Supply Temperature,True,1,"[16, 39, 1, 4, 98]","[water, leaving, sensor, temp, evaporator]",None
649,756,Water Supply Pressure,True,3,"[16, 39, 1, 6]","[water, leaving, sensor, pressure]",None
650,757,Water Return Pressure,True,3,"[16, 40, 1, 6]","[water, entering, sensor, pressure]",None


### Haystack Tag Definitions

In [7]:
# Get all tags and their definitions from the Data Model
pd.DataFrame(client.get_tags())

,id,name,definition,def_source,def_url,category
0,120,battery,A container that stores chemical energy that c...,brick,https://brickschema.org/ontology/1.1/classes/B...,None
1,191,exhaustVAV,A device that regulates the volume of air bein...,onboard,None,None
2,193,oil,"A viscous liquid derived from petroleum, espec...",brick,https://brickschema.org/ontology/1.2/classes/Oil/,None
3,114,fumeHood,A fume-collection device mounted over a work s...,brick,https://brickschema.org/ontology/1.1/classes/F...,None
4,118,limit,A parameter that places a lower or upper bound...,brick,https://brickschema.org/ontology/1.1/classes/L...,Point Class
...,...,...,...,...,...,...
238,12,speed,When used as a vfd point it is speed measured ...,onboard,None,None
239,67,makeup,Points associated with the flow of makeup air;...,onboard,None,None
240,60,max,Associated with a maximum value.,onboard,None,Quantity Modifier
241,61,min,Associated with a minumum value.,onboard,None,Quantity Modifier


# `2.0 ACTUAL BUILDING DATA`

### Buildings

In [8]:
buildings = pd.json_normalize(client.get_all_buildings())
buildings.head(1).T

,0
id,441
org_id,5
name,88737
address,None
sq_ft,551143.0
image_src,None
bms_manufacturer,None
bms_product_name,None
bms_version,None
timezone,America/New_York


In [9]:
bd = buildings[['id', 'name', 'equip_count', 'point_count','sq_ft', 'info.geoCity','info.customerType']]
bd = bd.rename(columns={'id':'building_id', 'name':'building_name', 'info.geoCity':'city', 'info.customerType':'building_type'})
bd

,building_id,building_name,equip_count,point_count,sq_ft,city,building_type
0,441,88737,20,1293,551143.0,Bronx,Multifamily
1,140,83483,13,354,21869.0,New York,Commercial Office
2,191,94414,7,19,198057.0,Victor,Commercial Retail
3,231,89496,4,51,NaN,New York,
4,248,116742,7,34,127000.0,New York,Multifamily
...,...,...,...,...,...,...,...
224,353,94413,7,19,198057.0,Henrietta,Commercial Retail
225,370,95457,5,22,421316.0,New York,Hospitality
226,383,79528,11,362,75000.0,Purchase,Commercial Office
227,398,109188,26,295,860000.0,New York,Commercial Office


### Equipments

In [10]:
# Get a list of all the equipment in your building
equipment = pd.json_normalize(client.get_all_equipment()).rename(columns={'tags':'equip_tags'})

In [11]:
# equipment.info()

In [12]:
equipment.head(1).T

,0
id,28797
building_id,441
equip_id,boiler-1
suffix,1
equip_type_name,Boiler
equip_type_id,19
equip_type_abbr,BLR
equip_type_tag,boiler
equip_subtype_name,None
equip_subtype_id,NaN


In [13]:
sub_type = pd.DataFrame(equip_type[equip_type.tag_name == 'fan']['sub_types'].item())
sub_type

,id,equipment_type_id,tag_name,name_long,name_abbr
0,12,26,exhaustFan,Exhaust Fan,EFN
1,13,26,reliefFan,Relief Fan,RlFN
2,14,26,returnFan,Return Fan,RFN
3,15,26,supplyFan,Supply Fan,SFN
4,27,26,boosterFan,Booster Fan,BFN
5,28,26,coolingTowerFan,Cooling Tower Fan,CTFN


### Points

In [14]:
# load data and rename columns
points = pd.json_normalize(client.get_all_points())
points = points.rename(columns={'id':'point_id',
 'name':'point_name',
 'description':'point_description',
 'value':'point_value',
 'type':'point_type'})

In [15]:
# drop non-meaningful columns
points.drop(columns=['device',
 'network_device',
 'objectId',
 'datasource_hash',
 'state_text',
 'state_text.1',
 'state_text.2',
 'state_text.3',
 'state_text.4',
 'state_text.5'], inplace=True)

In [16]:
# drop na
print('before dropna: ', points.shape[0])
points.drop(points[points.first_updated.isnull() == True].index, axis=0, inplace=True)
print('after dropna: ', points.shape[0])

before dropna:  44390
after dropna:  44390


## Time Stamps

In [17]:
points['first_updated'] = points.first_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc))
points['first_updated'].min()

Timestamp('2015-11-19 19:15:00+0000', tz='UTC')

In [18]:
points['last_updated'] = points.last_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc))
points['last_updated'].max()

Timestamp('2022-01-06 23:45:00+0000', tz='UTC')

# `3.0 METADATA CSV OUTPUT`

In [19]:
metadata_0 = pd.merge(points, bd, how='left', on='building_id')
metadata_0.describe(include='all', datetime_is_numeric=False).T

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
point_id,44390.0,NaN,NaN,NaN,NaT,NaT,256169.595247,61628.538478,190804.0,204291.25,228462.5,283887.75,427779.0
building_id,44390.0,NaN,NaN,NaN,NaT,NaT,323.11444,127.35999,98.0,190.0,375.0,420.0,503.0
last_updated,44390,2160,2018-12-17 17:45:00+00:00,4827,2015-11-19 19:15:00+00:00,2022-01-06 23:45:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first_updated,44390,1890,2018-02-01 05:00:00+00:00,3262,2015-11-19 19:15:00+00:00,2021-12-12 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
point_name,44390,14284,,29732,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
point_description,31843,30772,Voltage,42,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
units,37336,42,F,12610,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tagged_units,14654,32,f,5757,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
raw_unit_id,44390.0,NaN,NaN,NaN,NaT,NaT,11.06515,10.021779,2.0,2.0,5.0,20.0,70.0
point_value,44390,6008,0.0,4784,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
metadata = pd.merge(metadata_0, equipment[["id","equip_type_abbr","equip_type_name", "equip_tags"]], how="left", left_on="equip_id", right_on="id").drop(columns='id')
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44390 entries, 0 to 44389
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   point_id           44390 non-null  int64              
 1   building_id        44390 non-null  int64              
 2   last_updated       44390 non-null  datetime64[ns, UTC]
 3   first_updated      44390 non-null  datetime64[ns, UTC]
 4   point_name         44390 non-null  object             
 5   point_description  31843 non-null  object             
 6   units              37336 non-null  object             
 7   tagged_units       14654 non-null  object             
 8   raw_unit_id        44390 non-null  int64              
 9   point_value        44390 non-null  object             
 10  point_type         44390 non-null  object             
 11  point_type_id      44390 non-null  int64              
 12  measurement_id     44390 non-null  int64      

In [21]:
metadata.head(1).T

,0
point_id,310013
building_id,441
last_updated,2019-06-14 04:05:00+00:00
first_updated,2019-02-18 15:35:00+00:00
point_name,225 E 149th St. Boiler-1 Fire Rate Out
point_description,None
units,?
tagged_units,?
raw_unit_id,23
point_value,43


In [22]:
metadata.to_csv('/kaggle/working/metadata.csv', index=False)
print('exported to CSV!')

exported to CSV!


In [23]:
metadata.shape

(44390, 24)

# `>> THE END --> BELOW IS DATA FILTERING`

In [24]:
equip_type[equip_type.tag_name.str.lower() == 'heatpump']

,id,tag_name,name_long,name_abbr,active,flow_order,critical_point_types,sub_types,tags
10,29,heatPump,Heat Pump,HP,True,1,"[79, 77, 122, 121]",[],"[hvac, dxHeating, dxCooling, heatPump]"


In [25]:
point_type[point_type.id.isin([79, 77, 122, 121])]

,id,tag_name,active,measurement_id,tag_set_ids,tags,default_unit_id
25,122,Zone Temperature Setpoint,True,1,"[15, 2, 4, 62]","[air, sp, temp, zone]",None
34,79,Discharge Air Temperature,True,1,"[15, 36, 1, 4]","[air, discharge, sensor, temp]",None
38,77,Zone Temperature,True,1,"[15, 1, 4, 62]","[air, sensor, temp, zone]",None
92,121,Status,True,11,"[19, 1]","[run, sensor]",None


In [26]:
point_type[point_type.tag_name.str.contains('Cooling Output')]

,id,tag_name,active,measurement_id,tag_set_ids,tags,default_unit_id
229,327,Cooling Output,True,23,"[1, 50]","[sensor, cooling]",None


# `4.0 QUERY DATA`

`Select Time Window:`

In [27]:
# SELECT TIME
# __________________________________________________
tz = pytz.timezone('UTC')
# Select your date and time (time is optional)
start = datetime(2019,1,1,0,0,0).replace(tzinfo=tz)
end = datetime(2020,1,1,0,0,0).replace(tzinfo=tz)

`Get Points:`

In [28]:
metadata.building_type.value_counts()

Commercial Office     10361
Healthcare             8647
K-12 School            7076
Multifamily            5834
Not For Profit         3799
Hospitality            3072
Commercial Retail      1625
Food/Beverage          1444
College/University      992
Chemicals               195
                         62
Manufacturing            30
Name: building_type, dtype: int64

In [29]:
# metadata['days'] = metadata.last_updated - metadata.first_updated
# tmp = metadata.sort_values('days', ascending=False)
# tmp[(tmp.days > '500 days') & (tmp.last_updated > '2021-10') ].groupby('building_id').max().sort_values('building_type')

In [30]:
metadata[(metadata.building_type == 'Multifamily') & (metadata.city == 'New York') & (metadata.last_updated >'2019-08-01') & (metadata.first_updated <'2019-04-01') & (metadata.point_type=='Zone Temperature')]

,point_id,building_id,last_updated,first_updated,point_name,point_description,units,tagged_units,raw_unit_id,point_value,...,equip_id,building_name,equip_count,point_count,sq_ft,city,building_type,equip_type_abbr,equip_type_name,equip_tags
19167,273926,421,2019-08-27 20:15:00+00:00,2019-03-01 04:45:00+00:00,,215 E 59th Street Lobby temp,F,None,2,72.83,...,26335,108774,2,30,7960.0,New York,Multifamily,SITE,Site Sensors,[site]
19168,273925,421,2019-08-27 20:15:00+00:00,2019-03-01 04:45:00+00:00,,215 E 59th Street Apt 4F Loft temp,F,None,2,82.17,...,26335,108774,2,30,7960.0,New York,Multifamily,SITE,Site Sensors,[site]
19169,273924,421,2019-08-27 20:15:00+00:00,2019-03-01 04:45:00+00:00,,215 E 59th Street Apt 4F 2 temp,F,None,2,72.72,...,26335,108774,2,30,7960.0,New York,Multifamily,SITE,Site Sensors,[site]
19170,273923,421,2019-08-27 20:15:00+00:00,2019-03-01 04:45:00+00:00,,215 E 59th Street Apt 4F 1 temp,F,None,2,74.30,...,26335,108774,2,30,7960.0,New York,Multifamily,SITE,Site Sensors,[site]
19171,273922,421,2019-08-27 20:15:00+00:00,2019-03-01 04:45:00+00:00,,215 E 59th Street Apt 3F 2 temp,F,None,2,73.96,...,26335,108774,2,30,7960.0,New York,Multifamily,SITE,Site Sensors,[site]
19172,273921,421,2019-08-27 20:15:00+00:00,2019-03-01 04:45:00+00:00,,215 E 59th Street Apt 3F 1 temp,F,None,2,73.73,...,26335,108774,2,30,7960.0,New York,Multifamily,SITE,Site Sensors,[site]
19173,273920,421,2019-08-27 20:15:00+00:00,2019-03-01 04:45:00+00:00,,215 E 59th Street Apt 2F 2 temp,F,None,2,69.57,...,26335,108774,2,30,7960.0,New York,Multifamily,SITE,Site Sensors,[site]
19175,273918,421,2019-08-27 20:15:00+00:00,2019-03-01 04:45:00+00:00,,215 E 59th Street 1st Floor Shop temp,F,None,2,72.05,...,26335,108774,2,30,7960.0,New York,Multifamily,SITE,Site Sensors,[site]
19196,273927,421,2019-08-27 20:15:00+00:00,2019-03-01 04:45:00+00:00,,215 E 59th Street Boiler Aquastat Temperature,F,None,2,90.41,...,26336,108774,2,30,7960.0,New York,Multifamily,BLR,Boiler,"[boiler, hvac]"
40459,227099,371,2019-09-20 04:15:00+00:00,2019-03-19 03:45:00+00:00,,150 East 34th Street Boiler Plant BoilerRoomTemp,F,None,2,98.32,...,22264,95480,7,64,176584.0,New York,Multifamily,HWS,Hot Water Plant,"[hvac, hotWaterPlant]"


In [31]:
# tmp[(tmp.point_name.str.contains('Total')) & (tmp.days > '300 days') &  tmp.point_type.str.contains('Consumption')]['point_id'].to_list()

In [32]:
metadata.city

0           Bronx
1           Bronx
2           Bronx
3           Bronx
4           Bronx
           ...   
44385    New York
44386    New York
44387    New York
44388    New York
44389    New York
Name: city, Length: 44390, dtype: object

In [33]:
# SELECT POINTS
# __________________________________________________
points = metadata[(metadata.building_type == 'Multifamily') & (metadata.city == 'New York') & (metadata.last_updated >'2019-12-01') & (metadata.point_type=='Zone Temperature')]['point_id'].to_list() #(metadata.point_type_id.isin([118,119,146])) & 

print(f"Points extracting from {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}: \n")
print(np.array(points))

Points extracting from 2019-01-01 to 2020-01-01: 

[217159 217158 217157 217156 217155 217154 217153 217152 217151 217150
 326032 326031 326030 326029 326028 326027 326026 326025 326024 326023
 201013 201011 201014 201010 201012 201008 201009 203669 203668 203667
 203666 203665 203664 203663 203662 203661 203660 220503 220508 220507
 220506 220504 220505 199172 199170 199169 199167 199166 199171 199164
 199163 199168 199165 427763 427762 427761 427760 427759 427758 427757
 427756 427755 427754 427753 427752 427751 427750 427749 427748 427747
 427746 427745 427744 427743 427742 205007 205005 205004 205003 205002
 205001 205000 204999 204998 204997 204996 204995 204994 204993 204992
 204991 204990 204989 204988 204987 204986 204985 204984]


`Get Time Series:`

In [34]:
points=['283383']

In [35]:
# REQUEST TIME SERIES DATA
# __________________________________________________
print('\n query start time: ', datetime.today())

# Ask for points within a time range
timeseries_query = TimeseriesQuery(point_ids = points, start = start, end = end)
# Exceute query (will return and object)
query_results = client.stream_point_timeseries(timeseries_query)
# Convert to dataframe
data = points_df_from_streaming_timeseries(query_results)
# query results
print(' query end time: ', datetime.today())
print('\n')
display(data)
print('\nPoints Stats:')
data.describe()


 query start time:  2022-05-25 00:19:47.828164
 query end time:  2022-05-25 00:19:48.451814




,timestamp,283383
0,2019-12-01T05:00:00Z,61.43
1,2019-12-01T05:15:00Z,61.38
2,2019-12-01T05:30:00Z,61.30
3,2019-12-01T05:45:00Z,61.94
4,2019-12-01T06:00:00Z,61.32
...,...,...
2917,2019-12-31T14:15:00Z,61.31
2918,2019-12-31T14:30:00Z,61.35
2919,2019-12-31T14:45:00Z,61.30
2920,2019-12-31T15:00:00Z,61.29



Points Stats:


,283383
count,2922.000000
mean,63.695828
std,4.525706
min,0.000000
25%,61.550000
50%,63.010000
75%,66.047500
max,76.090000


`Export Time series to CSV: `

In [36]:
# export CSV
data.to_csv(f"/kaggle/working/heat_pump_zone_temp_hospitality_time_series.csv", index=False)

In [37]:
metadata[(metadata.building_id == 426) & (metadata.point_type == 'Zone Temperature') & (metadata.equip_type_name == 'Heat Pump')]

,point_id,building_id,last_updated,first_updated,point_name,point_description,units,tagged_units,raw_unit_id,point_value,...,equip_id,building_name,equip_count,point_count,sq_ft,city,building_type,equip_type_abbr,equip_type_name,equip_tags
4030,283383,426,2020-12-01 04:45:00+00:00,2017-12-01 05:00:00+00:00,480 W 42nd St - Signature Theatre WSHP5 Space ...,None,F,f,2,71.32,...,26958,76513,92,959,80000.0,New York,Hospitality,HP,Heat Pump,"[dxCooling, dxHeating, heatPump, hvac]"
